In [4]:
import os
import duckdb
import time
from dotenv import load_dotenv
from sql_stm import DUCKDB_CONF
import boto3
load_dotenv()
conn = duckdb.connect()
client = boto3.client("athena")

conf = {
        "host": os.getenv("host"),
        "port": 3306,
        "user": os.getenv("user"),
        "password": os.getenv("password"),
        "database": "demo_db",
    }
conn.query(DUCKDB_CONF.format(**conf))

In [5]:
df_count = conn.query("select count(*) from demo_db.exchange_rate").df()
total_rows = df_count.iloc[0, 0]
print(total_rows)

344


In [6]:
query_template = """
insert into awsdatacatalog.raw.exchange_rate (num_code, char_code, unit, currency, rate, business_date, surrogate_key, primary_key, partition_name)
select
num_code, char_code, unit, currency, rate, business_date, surrogate_key, primary_key, partition_name
from Finance.demo_db.exchange_rate a
where not exists(select 1 from awsdatacatalog.raw.exchange_rate b where b.surrogate_key = a.surrogate_key)
order by surrogate_key
limit {limit}
"""
# run query for batch with 200 records each
batch = 0
limit = 100
def execute_query_with_batch(query_template, batch, limit):
    qry_stm = query_template.format(business_date="{business_date}", limit=limit)
    res = client.start_query_execution(QueryString=qry_stm, QueryExecutionContext={"Database": "awsdatacatalog"})
    query_id = res["QueryExecutionId"]
    print(f"Query {query_id} submitted, batch {batch} to {batch + limit}")
    while not client.get_query_execution(QueryExecutionId=query_id)["QueryExecution"]["Status"]["State"] in ["SUCCEEDED"]:
        time.sleep(1)
    print(f"Query {query_id} succeeded")

while batch <= total_rows:
    execute_query_with_batch(query_template, batch, limit)
    batch += limit
    print(f"Batch {batch} completed")

Query 220a69db-1a09-45ec-a3c7-b79fc41326c1 submitted, batch 0 to 100
Query 220a69db-1a09-45ec-a3c7-b79fc41326c1 succeeded
Batch 100 completed
Query 099111bc-16e0-48c7-aa17-be1fa282db51 submitted, batch 100 to 200
Query 099111bc-16e0-48c7-aa17-be1fa282db51 succeeded
Batch 200 completed
Query c78e0966-01d2-47fc-bf45-aaabf80ab229 submitted, batch 200 to 300
Query c78e0966-01d2-47fc-bf45-aaabf80ab229 succeeded
Batch 300 completed
Query d4e3526d-2f8c-473d-a39e-08ffffdbc5cc submitted, batch 300 to 400
Query d4e3526d-2f8c-473d-a39e-08ffffdbc5cc succeeded
Batch 400 completed
